In [958]:
import pandas as pd
import cufflinks as cf
import warnings
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chisquare
from  functools import reduce
import re
from plotly.offline import plot,iplot
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px#graficos express
import sys
warnings.filterwarnings("ignore")
cf.go_offline()
pd.set_option("display.max_columns",200)

In [959]:
def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO(La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO(La porporción de categorias es la misma que la general)")

In [960]:
def other(df,col,n):
    aux=list(df[col].value_counts(1).index[n:].values)
    dictio_aux=dict(zip(aux,["otro"]*len(aux)))
    df[col]=df[col].replace(dictio_aux)
    return df

def multiple_replace(string, rep_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(rep_dict,key=len,reverse=True)]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)]+"_", string)

In [961]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=True):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
        dictionary: Dictionary of column names (keys) and GVIF ** (1 / (2 * Df)) ** 2 (values)
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final,gvif_filter,df_m


In [962]:
sys.path.append("../libreria/")

In [963]:
from libreria import rename_cols,completitud,text_clean,box,histogram

In [964]:
df=pd.read_csv("../datos/dataset_house_prices.csv")

## Variables

**El problema**

En análisis exploratorio tiene el objetivo de entender el significado y la relevancia de cada variable con respecto al problema. Además apartir de este análisis se pueden determinar que acciones tomar para cada variable En este caso:

- La relevancia de la variable en la compra de una casa.
- La importancia de la variable.
- Solapamiento con otras variables.


Variable objetivo : 'SalePrice'.

- MSSubClass: clase de construcción
- MSZoning: clasificación de la zona
- LotFrontage: pies lineales de calle de la parcela
- LotArea: tamaño de la parcela en pies cuadrados
- Street: tipo de acceso por carretera
- Alley: tipo de acceso al callejón
- LotShape: forma de la parcela
- LandContour: planitud de la parcela
- Utilities: servicios públicos disponibles
- LotConfig: Configuración de parcela
- LandSlope: pendiente de la parcela
- Neighborhood: ubicación física dentro de los límites de la ciudad de Ames
- Condition1: proximidad a la carretera principal o al ferrocarril
- Condition2: proximidad a la carretera principal o al ferrocarril (si hay un segundo)
- BldgType: tipo de vivienda
- HouseStyle: estilo de vivienda
- OverallQual: calidad general del material y del acabado
- OverallCond: condición general
- YearBuilt: fecha original de construcción
- YearRemodAdd: fecha de remodelación
- RoofStyle: tipo de cubierta
- RoofMatl: material del techo
- Exterior1st: revestimiento exterior de la casa
- Exterior2nd: revestimiento exterior de la casa (si hay más de un material)
- MasVnrType: tipo de revestimiento de mampostería
- MasVnrArea: área de revestimiento de mampostería en pies cuadrados
- ExterQual: calidad del material exterior
- ExterCond: estado del material en el exterior
- Foundation: tipo de cimentación
- BsmtQual: altura del sótano
- BsmtCond: estado general del sótano
- BsmtExposure: paredes del sótano a nivel de calle o de jardín
- BsmtFinType1: calidad del área acabada del sótano
- BsmtFinSF1: pies cuadrados de la superficie acabada tipo 1
- BsmtFinType2: calidad de la segunda superficie acabada (si existe)
- BsmtFinSF2: Pies cuadrados de la superficie acabada tipo 2
- BsmtUnfSF: pies cuadrados del área sin terminar del sótano
- TotalBsmtSF: pies cuadrados totales del sótano
- Heating: tipo de calefacción
- HeatingQC: calidad y estado de la calefacción
- CentralAir: aire acondicionado central
- Electrical: sistema eléctrico
- 1erFlrSF: área en pies cuadrados de la primera planta (o planta baja)
- 2ndFlrSF: área en pies cuadrados de la segunda planta
- LowQualFinSF: pies cuadrados acabados de baja calidad (todos los pisos)
- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- BsmtFullBath: cuartos de baño completos en el sótano
- BsmtHalfBath: medio baño del sótano
- FullBath: baños completos sobre el nivel del suelo
- HalfBath: medios baños sobre el nivel del suelo
- Bedroom: número de dormitorios por encima del nivel del sótano
- Kitchen: número de cocinas
- KitchenQual: calidad de la cocina
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- Functional: valoración de la funcionalidad de la vivienda
- Fireplaces: número de chimeneas
- FireplaceQu: calidad de la chimenea
- GarageType: ubicación del garaje
- GarageYrBlt: año de construcción del garaje
- GarageFinish: acabado interior del garaje
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados
- GarageQual: calidad de garaje
- GarageCond: condición de garaje
- PavedDrive: calzada asfaltada
- WoodDeckSF: area de plataforma de madera en pies cuadrados
- OpenPorchSF: área de porche abierto en pies cuadrados
- EnclosedPorch: área de porche cerrada en pies cuadrados
- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados
- PoolQC: calidad de la piscina
- Fence: calidad de la valla
- MiscFeature: característica miscelánea no cubierta en otras categorías
- MiscVal: valor en dólares de la característica miscelánea
- MoSold: mes de venta
- YrSold: año de venta
- SaleType: tipo de venta
- SaleCondition: Condiciones de venta

In [965]:
v_feats=["MSSubClass","MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Condition1","Condition2","BldgType","HouseStyle","OverallQual","OverallCond",
        "YearBuilt","YearRemodAdd","RoofStyle","RoofMatl","MasVnrType","ExterQual",
        "ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
        "Heating","HeatingQC","CentralAir","Electrical","LowQualFinSF","BsmtHalfBath","FullBath","KitchenAbvGr","KitchenQual","Functional","Fireplaces","FireplaceQu",
        "GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence",
        "MiscFeature","MoSold","YrSold","SaleType","SaleCondition"]
c_feats=["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF",
        "2ndFlrSF","GrLivArea","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch",
        "PoolArea","MiscVal","BedroomAbvGr","HalfBath","GarageCars","BsmtFullBath","TotRmsAbvGrd"]
tgt=["SalePrice"]

df=rename_cols(df,v_feats,"v_")
df=rename_cols(df,c_feats,"c_")
df=rename_cols(df,tgt,"tgt_")

In [966]:
#Eliminamos aquellas variables que que hacen referencia a la venta
df.drop(columns=["v_SaleType","v_SaleCondition"],inplace=True)

In [967]:
df.drop(columns=["Neighborhood","Exterior2nd","Exterior1st","v_YrSold","v_MoSold"],inplace=True)

In [968]:
#NO hay duplicados
df[df.duplicated(subset=["Id"])]

,Id,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_Alley,v_LotShape,v_LandContour,v_Utilities,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_FireplaceQu,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,v_PoolQC,v_Fence,v_MiscFeature,c_MiscVal,tgt_SalePrice


In [969]:
#Variables para limpiar el texto
features=list(filter(lambda x: x not in ["v_GarageYrBlt","v_YearBuilt"],df.filter(like="v_") ))


for col in features:
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)

In [970]:
histogram(df,"tgt_SalePrice",5,"tgt_SalePrice").show()

In [971]:
miss=completitud(df)

In [972]:
miss

,columna,total,completitud
0,v_PoolQC,1453,0.479452
1,v_MiscFeature,1406,3.698630
2,v_Alley,1369,6.232877
3,v_Fence,1179,19.246575
4,v_FireplaceQu,690,52.739726
...,...,...,...
69,c_MiscVal,0,100.000000
70,v_HeatingQC,0,100.000000
71,v_CentralAir,0,100.000000
72,c_2ndFlrSF,0,100.000000


In [973]:
null_cols=list(miss[miss["completitud"]<80]["columna"].values)

In [974]:
null_cols

['v_PoolQC', 'v_MiscFeature', 'v_Alley', 'v_Fence', 'v_FireplaceQu']

In [975]:
df=df.drop(columns=null_cols)

In [976]:
df["v_MSSubClass"]=df["v_MSSubClass"].astype(int)

In [977]:
df_multi=df.copy()

In [978]:
df_multi=df.copy()

## Isolation Forest

**La clase sklearn.ensemble.IsolationForest incorpora las principales funcionalidades que se necesitan a la hora de trabajar con modelos Isolation Forest. Los principales argumentos para entrenar este tipo de modelos son:**

- n_estimators: número de árboles que forman el modelo.

- max_samples: número de observaciones empleadas para entrenar cada árbol.

- max_features : El número de características que se extraerán de X para entrenar cada estimador base.

- contamination: proporción de anomalías esperadas en los datos de entrenamiento. En base a este valor, se establece el límite acorde al cual se clasifican las observaciones en normales o anómalas.

- random_state: semilla para garantizar la reproducibilidad de los resultados.

- Se procede a entrenar un modelo asumiendo que hay un 1% de observaciones anómalas en el conjunto de entrenamiento.

In [979]:
from sklearn.ensemble import IsolationForest

In [980]:
df=df.drop(columns=["Id"])

In [981]:
df.shape

(1460, 68)

In [982]:
df_iso=df.filter(like="c_").dropna()

In [983]:
#max_features=1
max_features=df_iso.shape[1]
n_estimators=50
max_samples='auto'
contamination=float(0.2)
model=IsolationForest(max_features = max_features, n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
model.fit(df_iso)

IsolationForest(contamination=0.2, max_features=23, n_estimators=50)

- Los modelos IsolationForest tienen dos métodos de predicción con los que se obtiene distinta información.

- Con el método predict() se devuelve directamente la clasificación de anomalía (-1) o no anomalía (1) acorde a la proporción de contaminación que se ha indicado en la definición del modelo.

In [984]:
df_iso["outlier"]=model.predict(df_iso)

In [985]:
df_iso["outlier"].value_counts()

 1    956
-1    239
Name: outlier, dtype: int64

In [986]:
df_iso[df_iso["outlier"]==-1].shape[0]/df_iso.shape[0]

0.2

In [987]:
indices=list(df_iso[df_iso["outlier"]==1].index)

In [988]:
df=df.iloc[indices]

In [989]:
df.reset_index(drop=True,inplace=True)

## MISSINGS

### Variables unitarias

In [990]:
#Eliminamos Variables unarias

In [991]:
df[["v_Utilities","v_Street"]].describe()

,v_Utilities,v_Street
count,956,956
unique,1,2
top,allpub,pave
freq,956,953


In [992]:
df.drop(columns=["v_Utilities"],inplace=True)

In [993]:
miss=completitud(df)

miss_cols=list(miss[miss["completitud"]<100]["columna"].values)

In [994]:
miss

,columna,total,completitud
0,v_GarageQual,35,96.338912
1,v_GarageType,35,96.338912
2,v_GarageYrBlt,35,96.338912
3,v_GarageFinish,35,96.338912
4,v_GarageCond,35,96.338912
...,...,...,...
62,c_BsmtUnfSF,0,100.000000
63,c_TotalBsmtSF,0,100.000000
64,c_MiscVal,0,100.000000
65,v_YearBuilt,0,100.000000


In [995]:
miss_cols

['v_GarageQual',
 'v_GarageType',
 'v_GarageYrBlt',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtFinType2',
 'v_BsmtQual',
 'v_BsmtFinType1',
 'v_BsmtCond',
 'v_Electrical']

### NORMALIZAMOS DISCRETAS

In [996]:
v_=list(filter(lambda x:"v_" in x,miss_cols))
v_=[x for x in v_ if x!="v_GarageYrBlt"]

In [997]:
for col in v_:
    print(col)
    print(df[col].isnull().sum())
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)
    print(df[col].isnull().sum())

v_GarageQual
35
35
v_GarageType
35
35
v_GarageFinish
35
35
v_GarageCond
35
35
v_BsmtExposure
20
20
v_BsmtFinType2
19
19
v_BsmtQual
19
19
v_BsmtFinType1
19
19
v_BsmtCond
19
19
v_Electrical
1
1


In [998]:
for col in v_:
    print(col)
    display(df[col].value_counts(1))
    print("\n")

v_GarageQual


ta    0.938111
fa    0.045603
gd    0.010858
po    0.003257
ex    0.002172
Name: v_GarageQual, dtype: float64



v_GarageType


attchd     0.601520
detchd     0.331162
builtin    0.044517
basment    0.013029
carport    0.006515
2types     0.003257
Name: v_GarageType, dtype: float64



v_GarageFinish


...

unf    0.331910
glq    0.258271
alq    0.154749
blq    0.103522
rec    0.102455
lwq    0.049093
Name: v_BsmtFinType1, dtype: float64



v_BsmtCond


ta    0.919957
gd    0.043757
fa    0.035219
po    0.001067
Name: v_BsmtCond, dtype: float64



v_Electrical


sbrkr    0.900524
fusea    0.073298
fusef    0.021990
fusep    0.003141
mix      0.001047
Name: v_Electrical, dtype: float64

In [999]:
##"v_GarageType"
##"v_Electrical"
#MasVnrType

In [1000]:
dictionary={"v_BldgType":1,"v_Foundation":2,"v_Functional":1,"v_Heating":1,"v_HouseStyle":3,"v_LandContour":1,
"v_LandSlope":1,"v_LotConfig":2,"v_LotShape":1,"v_MSZoning":2,"v_RoofMatl":1,"v_RoofStyle":2,"v_Condition2":1,"v_Condition1":1,"v_BsmtFinType2":1,"v_BsmtFinType1":3,"v_BsmtExposure":1}

In [1001]:
for i,j in dictionary.items():
    print(i)
    df=other(df,i,j)
    print(df[i].value_counts())

v_BldgType
1fam    809
otro    147
Name: v_BldgType, dtype: int64
v_Foundation
pconc     416
cblock    407
otro      133
Name: v_Foundation, dtype: int64
v_Functional
typ     892
otro     64
Name: v_Functional, dtype: int64
v_Heating
gasa    935
otro     21
Name: v_Heating, dtype: int64
v_HouseStyle
1story    486
2story    273
1 5fin    115
otro       82
Name: v_HouseStyle, dtype: int64
v_LandContour
lvl     873
otro     83
Name: v_LandContour, dtype: int64
v_LandSlope
gtl     924
otro     32
Name: v_LandSlope, dtype: int64
v_LotConfig
inside    743
corner    154
otro       59
Name: v_LotConfig, dtype: int64
v_LotShape
reg     707
otro    249
Name: v_LotShape, dtype: int64
v_MSZoning
rl      725
rm      164
otro     67
Name: v_MSZoning, dtype: int64
v_RoofMatl
compshg    950
otro         6
Name: v_RoofMatl, dtype: int64
v_RoofStyle
gable    782
hip      160
otro      14
Name: v_RoofStyle, dtype: int64
v_Condition2
norm    947
otro      9
Name: v_Condition2, dtype: int64
v_Condition1
no

In [1002]:
dictio_GarageType=dict(zip(list(df["v_GarageType"].value_counts().index[-3:]),["otro"]*3))
dictio_electrical=dict(zip(list(df["v_Electrical"].value_counts().index[-3:]),["otro"]*3))
dictio_MasVnrType=dict(zip(list(df["v_MasVnrType"].value_counts().index[-2:]),["otro"]*2))

In [1003]:
df["v_GarageType"]=df["v_GarageType"].replace(dictio_GarageType)
df["v_Electrical"]=df["v_Electrical"].replace(dictio_electrical)
df["v_MasVnrType"]=df["v_MasVnrType"].replace(dictio_MasVnrType)

In [1004]:
df["v_GarageType"].value_counts(1)

attchd     0.601520
detchd     0.331162
builtin    0.044517
otro       0.022801
Name: v_GarageType, dtype: float64

In [1005]:
df["v_Electrical"].value_counts(1)

sbrkr    0.900524
fusea    0.073298
otro     0.026178
Name: v_Electrical, dtype: float64

In [1006]:
## "v_GarageQual","v_GarageCond"

In [1007]:
df["v_GarageQual"].value_counts(1)

ta    0.938111
fa    0.045603
gd    0.010858
po    0.003257
ex    0.002172
Name: v_GarageQual, dtype: float64

In [1008]:
df["v_GarageCond"].value_counts(1)

ta    0.952226
fa    0.034745
po    0.006515
gd    0.004343
ex    0.002172
Name: v_GarageCond, dtype: float64

In [1009]:
dictio_qual_cond=dict(zip(["ex","gd","ta","fa","po"],[3,3,2,1,1]))

In [1010]:
feats_replace_cond_qual=["v_GarageQual","v_GarageCond","v_BsmtCond","v_BsmtQual","v_ExterCond","v_ExterQual","v_HeatingQC","v_KitchenQual"]

In [1011]:
for feat in feats_replace_cond_qual:
    print(feat)
    df[feat]=df[feat].replace(dictio_qual_cond)
    print(df[feat].value_counts(1))
    print("\n")

v_GarageQual
2.0    0.938111
1.0    0.048860
3.0    0.013029
Name: v_GarageQual, dtype: float64


v_GarageCond
2.0    0.952226
1.0    0.041260
3.0    0.006515
Name: v_GarageCond, dtype: float64


v_BsmtCond
2.0    0.919957
3.0    0.043757
1.0    0.036286
Name: v_BsmtCond, dtype: float64


v_BsmtQual
2.0    0.485592
3.0    0.481323
1.0    0.033084
Name: v_BsmtQual, dtype: float64


v_ExterCond
2    0.884937
3    0.095188
1    0.019874
Name: v_ExterCond, dtype: float64


v_ExterQual
2    0.654812
3    0.335774
1    0.009414
Name: v_ExterQual, dtype: float64


v_HeatingQC
3    0.673640
2    0.289749
1    0.036611
Name: v_HeatingQC, dtype: float64


v_KitchenQual
2    0.532427
3    0.440377
1    0.027197
Name: v_KitchenQual, dtype: float64




In [1012]:
miss=completitud(df)
miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,35,96.338912
1,v_GarageType,35,96.338912
2,v_GarageYrBlt,35,96.338912
3,v_GarageFinish,35,96.338912
4,v_GarageCond,35,96.338912
5,v_BsmtExposure,20,97.907950
6,v_BsmtFinType2,19,98.012552
7,v_BsmtQual,19,98.012552
8,v_BsmtFinType1,19,98.012552
9,v_BsmtCond,19,98.012552


In [1013]:
#Remplazamos los valores de la fecha de construccion del garage por la fecha de construcción de la propiedad
df['v_GarageYrBlt']=df['v_GarageYrBlt'].fillna(df['v_YearBuilt']).astype(float)

In [1014]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,35,96.338912
1,v_GarageType,35,96.338912
2,v_GarageFinish,35,96.338912
3,v_GarageCond,35,96.338912
4,v_BsmtExposure,20,97.907950
5,v_BsmtQual,19,98.012552
6,v_BsmtFinType2,19,98.012552
7,v_BsmtCond,19,98.012552
8,v_BsmtFinType1,19,98.012552
9,v_Electrical,1,99.895397


### CATEGORICAS

In [1015]:
from sklearn.model_selection import train_test_split

In [1016]:
X_train,X_test=train_test_split(df,test_size=.2,random_state=413)

In [1017]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,35,96.338912
1,v_GarageType,35,96.338912
2,v_GarageFinish,35,96.338912
3,v_GarageCond,35,96.338912
4,v_BsmtExposure,20,97.907950
5,v_BsmtQual,19,98.012552
6,v_BsmtFinType2,19,98.012552
7,v_BsmtCond,19,98.012552
8,v_BsmtFinType1,19,98.012552
9,v_Electrical,1,99.895397


In [1018]:
v_feats=list(filter(lambda x:"v_" in x,list(miss[miss["completitud"]<100]["columna"].values)))

In [1019]:
v_feats

['v_GarageQual',
 'v_GarageType',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtQual',
 'v_BsmtFinType2',
 'v_BsmtCond',
 'v_BsmtFinType1',
 'v_Electrical']

In [1020]:
for column in v_feats:
    print(column)
    value=X_train[column].value_counts().index[0]
    print(value)
    print(chi_square(df,column,value))
    print("\n")

v_GarageQual
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageType
attchd
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageFinish
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtExposure
no
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtQual
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType2
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType1
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_Electrical
sbrkr
Aceptamos HO(La porporción de categorias es la misma que la general)
None




In [1021]:
from sklearn.impute import SimpleImputer

In [1022]:
imp=SimpleImputer(missing_values=np.nan,strategy="most_frequent")

In [1023]:
imp.fit(X_train[v_feats])

SimpleImputer(strategy='most_frequent')

In [1024]:
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

In [1025]:
X_train[v_feats]=pd.DataFrame(imp.transform(X_train[v_feats]),columns=v_feats)

In [1026]:
X_test[v_feats]=pd.DataFrame(imp.transform(X_test[v_feats]),columns=v_feats)

In [1027]:
miss=completitud(X_train)

miss[miss["completitud"]<100]

,columna,total,completitud


In [1028]:
miss=completitud(X_test)

miss[miss["completitud"]<100]

,columna,total,completitud


## Reduccion de dimensiones

In [1029]:
X_train["v_GarageYrBlt"]=X_train["v_GarageYrBlt"].astype(int)
X_test["v_GarageYrBlt"]=X_test["v_GarageYrBlt"].astype(int)
X_train["v_YearRemodAdd"]=X_train["v_YearRemodAdd"].astype(int)
X_test["v_YearRemodAdd"]=X_test["v_YearRemodAdd"].astype(int)

In [1030]:
X_train

,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal,tgt_SalePrice
0,90,rm,110.0,8472,grvl,otro,otro,corner,otro,otro,norm,otro,1story,5,5,1963,1963,gable,compshg,none,0.0,1,2,cblock,3.0,2.0,otro,otro,104,otro,712,0,816,gasa,2,n,sbrkr,816,0,0,816,1,0,1,0,2,1,2,5,typ,0,otro,1963,unf,2,516,2.0,2.0,y,106,0,0,0,0,0,0,110000
1,50,rm,52.0,6240,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,6,6,1934,1950,gable,compshg,none,0.0,2,2,pconc,2.0,2.0,no,unf,0,unf,0,816,816,gasa,2,y,sbrkr,816,0,360,1176,0,0,1,0,3,1,2,6,typ,1,detchd,1985,unf,2,528,2.0,2.0,y,112,0,0,0,0,0,400,114500
2,160,otro,24.0,2544,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,7,5,2005,2005,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,unf,0,unf,0,600,600,gasa,3,y,sbrkr,520,623,80,1223,0,0,2,1,2,1,3,4,typ,0,detchd,2005,rfn,2,480,2.0,2.0,y,0,166,0,0,0,0,0,147400
3,30,rl,80.0,11600,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,5,1922,1950,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0,unf,0,700,700,gasa,3,y,sbrkr,1180,0,0,1180,0,0,1,0,2,1,1,5,typ,1,detchd,1922,unf,1,252,2.0,1.0,y,0,0,67,0,0,0,0,137500
4,30,rm,60.0,9786,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,3,4,1922,1950,hip,compshg,none,0.0,2,2,otro,2.0,1.0,no,unf,0,unf,0,1007,1007,gasa,1,n,sbrkr,1077,0,0,1077,0,0,1,0,3,1,2,6,typ,1,detchd,1922,unf,1,210,2.0,1.0,p,0,100,48,0,0,0,0,91000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,70,rl,55.0,10592,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,7,1923,1996,hip,compshg,none,0.0,2,3,pconc,2.0,1.0,no,unf,0,unf,0,602,602,gasa,2,y,sbrkr,900,602,0,1502,0,0,1,1,3,1,3,7,typ,2,detchd,1923,unf,1,180,2.0,2.0,y,96,0,112,0,53,0,0,165000
760,60,rl,70.0,8750,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,5,1998,1998,gable,compshg,brkface,116.0,2,2,pconc,3.0,2.0,no,glq,505,unf,0,299,804,gasa,3,y,sbrkr,804,878,0,1682,0,0,2,1,3,1,3,7,typ,0,attchd,1998,rfn,2,523,2.0,2.0,y,0,77,0,0,0,0,0,200500
761,85,rl,69.0,10205,pave,otro,lvl,inside,gtl,norm,norm,1fam,otro,5,5,1962,1962,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,otro,otro,784,unf,0,141,925,gasa,2,y,sbrkr,999,0,0,999,1,0,1,0,3,1,2,6,typ,0,attchd,1962,unf,1,300,2.0,2.0,y,150,72,0,0,0,0,0,134500
762,60,rl,68.0,8935,pave,otro,lvl,inside,gtl,norm,norm,1fam,2story,7,5,2002,2002,gable,compshg,brkface,95.0,3,2,pconc,3.0,2.0,no,unf,0,unf,0,831,831,gasa,3,y,sbrkr,831,829,0,1660,0,0,2,1,3,1,3,7,typ,0,attchd,2002,rfn,2,493,2.0,2.0,y,144,68,0,0,0,0,0,195000


### Relación de valor perdido

In [1032]:
completitud(X_train)

,columna,total,completitud
0,v_MSSubClass,0,100.0
1,v_CentralAir,0,100.0
2,v_Electrical,0,100.0
3,c_1stFlrSF,0,100.0
4,c_2ndFlrSF,0,100.0
...,...,...,...
62,c_BsmtFinSF1,0,100.0
63,v_BsmtFinType2,0,100.0
64,c_BsmtFinSF2,0,100.0
65,v_YearRemodAdd,0,100.0


### Filtro de baja varianza

In [1033]:
X_train.filter(like="c_").describe(percentiles=np.arange(0.1,1.1,.1))

,c_LotFrontage,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,c_1stFlrSF,c_2ndFlrSF,c_GrLivArea,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_TotRmsAbvGrd,c_GarageCars,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal
count,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000
mean,67.176702,8883.808901,68.327225,379.238220,34.332461,570.674084,984.244764,1081.092932,307.938482,1392.912304,0.353403,0.341623,2.786649,6.242147,1.701571,448.650524,73.500000,35.965969,23.725131,3.184555,9.882199,0.848168,10.829843
std,19.709793,3201.077710,126.503830,382.827808,127.638002,388.374483,341.000408,302.061437,384.412530,385.782314,0.478340,0.474565,0.649855,1.299361,0.675307,183.634737,103.034007,48.612992,60.199475,29.860554,40.852344,23.443817,96.142573
min,21.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,483.000000,0.000000,520.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10%,43.000000,5000.000000,0.000000,0.000000,0.000000,108.000000,624.300000,756.000000,0.000000,910.600000,0.000000,0.000000,2.000000,5.000000,1.000000,240.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20%,52.000000,6787.200000,0.000000,0.000000,0.000000,216.000000,735.000000,832.000000,0.000000,1040.600000,0.000000,0.000000,2.000000,5.000000,1.000000,288.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30%,60.000000,7626.900000,0.000000,0.000000,0.000000,303.900000,815.900000,894.000000,0.000000,1141.900000,0.000000,0.000000,2.000000,6.000000,1.000000,352.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
40%,60.000000,8400.000000,0.000000,131.400000,0.000000,412.200000,874.400000,955.000000,0.000000,1247.200000,0.000000,0.000000,3.000000,6.000000,2.000000,420.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,66.000000,8874.500000,0.000000,329.000000,0.000000,529.000000,936.000000,1038.500000,0.000000,1352.000000,0.000000,0.000000,3.000000,6.000000,2.000000,460.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60%,70.000000,9524.000000,0.000000,464.800000,0.000000,638.000000,1026.000000,1102.200000,382.800000,1478.800000,0.000000,0.000000,3.000000,6.000000,2.000000,489.400000,49.000000,35.800000,0.000000,0.000000,0.000000,0.000000,0.000000


In [1034]:
X_train.drop(columns=["c_BsmtFullBath","c_PoolArea"],inplace=True)
X_test.drop(columns=["c_BsmtFullBath","c_PoolArea"],inplace=True)

In [1035]:
X_train.drop(columns=["c_MasVnrArea"],inplace=True)
X_test.drop(columns=["c_MasVnrArea"],inplace=True)

### Filtro de alta correlación

- La Correlación se utiliza para probar las relaciones entre variables cuantitativas o variables categóricas. En otras palabras, es una medida de cómo se relacionan las cosas. El estudio de cómo se correlacionan las variables se denomina análisis de correlación.
- Un coeficiente de correlación proporciona un resumen numérico del grado de asociación entre dos variables.
- Tipos:
- **Correlación r de Pearson: la correlación r de Pearson es el estadístico de correlación más utilizado para medir el grado de relación entre variables relacionadas linealmente. Por ejemplo, en el mercado de valores, si queremos medir cómo se relacionan dos acciones entre sí, se utiliza la correlación r de Pearson para medir el grado de relación entre las dos. La correlación punto-biserial se realiza con la fórmula de correlación de Pearson excepto que una de las variables es dicotómica.**
- **Correlación de rango de Kendall: la correlación de rango de Kendall es una prueba no paramétrica que mide la fuerza de la dependencia entre dos variables. Si consideramos dos muestras, ayb, donde cada tamaño de muestra es n, sabemos que el número total de emparejamientos con a b es n (n-1) / 2.**
- **Correlación de rango de Spearman: la correlación de rango de Spearman es una prueba no paramétrica que se utiliza para medir el grado de asociación entre dos variables. La prueba de correlación de rango de Spearman no conlleva ninguna suposición sobre la distribución de los datos y es el análisis de correlación apropiado cuando las variables se miden en una escala que es al menos ordinal.**

https://www.google.com/url?sa=i&url=https%3A%2F%2Fmedium.com%2F%40outside2SDs%2Fan-overview-of-correlation-measures-between-categorical-and-continuous-variables-4c7f85610365&psig=AOvVaw3KaiD8_vD816QPDiI7Uz8s&ust=1622782900559000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCIjvg8fX-vACFQAAAAAdAAAAABAD

In [1036]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados

In [1037]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")

In [1038]:
X_train.filter(like="v_").columns

Index(['v_MSSubClass', 'v_MSZoning', 'v_Street', 'v_LotShape', 'v_LandContour',
       'v_LotConfig', 'v_LandSlope', 'v_Condition1', 'v_Condition2',
       'v_BldgType', 'v_HouseStyle', 'v_OverallQual', 'v_OverallCond',
       'v_YearBuilt', 'v_YearRemodAdd', 'v_RoofStyle', 'v_RoofMatl',
       'v_MasVnrType', 'v_ExterQual', 'v_ExterCond', 'v_Foundation',
       'v_BsmtQual', 'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1',
       'v_BsmtFinType2', 'v_Heating', 'v_HeatingQC', 'v_CentralAir',
       'v_Electrical', 'v_LowQualFinSF', 'v_BsmtHalfBath', 'v_FullBath',
       'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional', 'v_Fireplaces',
       'v_GarageType', 'v_GarageYrBlt', 'v_GarageFinish', 'v_GarageQual',
       'v_GarageCond', 'v_PavedDrive'],
      dtype='object')

In [1040]:
corr=abs(corr)

In [1041]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


,c_BsmtUnfSF
c_BsmtUnfSF,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [1042]:
X_train.drop(columns=["c_TotRmsAbvGrd","c_GarageCars"],inplace=True)
X_test.drop(columns=["c_TotRmsAbvGrd","c_GarageCars"],inplace=True)

In [1043]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")
corr=abs(corr)

In [1044]:
X_train.drop(columns=["c_1stFlrSF"],inplace=True)
X_test.drop(columns=["c_1stFlrSF"],inplace=True)

In [1045]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


,c_BsmtUnfSF
c_BsmtUnfSF,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [1046]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

### Correlación (con el objetivo)

In [1047]:
feats=list(X_train.filter(like="c_").columns)+["tgt_SalePrice"]

In [1048]:
X_train[feats].corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

In [1049]:
corr=X_train[feats].corr(method="spearman")

In [1050]:
corr=abs(corr)

In [1051]:
corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False).style.background_gradient()

,tgt_SalePrice
tgt_SalePrice,1.000000
c_GrLivArea,0.689023
c_GarageArea,0.604712
c_TotalBsmtSF,0.549840
c_OpenPorchSF,0.484024
c_LotArea,0.370534
c_LotFrontage,0.357336
c_WoodDeckSF,0.335991
c_HalfBath,0.323333
c_EnclosedPorch,0.289811


In [1052]:
corr=corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False)

In [1053]:
cols_drop_low_corr=list(corr[corr["tgt_SalePrice"]<.1].index)

In [1054]:
cols_drop_low_corr

['c_BsmtFinSF2', 'c_ScreenPorch', 'c_3SsnPorch']

- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados

In [1055]:
X_train=X_train.drop(columns=cols_drop_low_corr)
X_test=X_test.drop(columns=cols_drop_low_corr)

### MULTICOLINEALIDAD

#### MULTICOLINEALIDAD CONTINUAS

In [1056]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [1057]:
#Función para multicolineaidad con continuas
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [1058]:
X_train.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)
X_test.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)

In [1059]:
for col in X_train.filter(like="c_").columns:
    X_train[col]=X_train[col].astype(float)

In [1060]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

In [1061]:
vif

,variables,VIF
0,c_LotFrontage,23.798841
1,c_LotArea,16.743894
2,c_BsmtFinSF1,20.593357
3,c_BsmtUnfSF,31.519864
4,c_TotalBsmtSF,93.661133
5,c_2ndFlrSF,8.837079
6,c_GrLivArea,67.279042
7,c_HalfBath,2.712329
8,c_BedroomAbvGr,22.693887
9,c_GarageArea,10.142397


In [1065]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False).head(50)

,variables,VIF
4,c_TotalBsmtSF,93.661133
0,c_LotFrontage,23.798841
1,c_LotArea,16.743894
6,c_GrLivArea,67.279042
9,c_GarageArea,10.142397
3,c_BsmtUnfSF,31.519864
2,c_BsmtFinSF1,20.593357
8,c_BedroomAbvGr,22.693887


In [1066]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [1067]:
vif_feats

['c_TotalBsmtSF',
 'c_LotFrontage',
 'c_LotArea',
 'c_GrLivArea',
 'c_GarageArea',
 'c_BsmtUnfSF',
 'c_BsmtFinSF1',
 'c_BedroomAbvGr']

In [1068]:
X_train.drop(columns=["c_TotalBsmtSF"],inplace=True)
X_test.drop(columns=["c_TotalBsmtSF"],inplace=True)

In [1069]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in ["tgt_SalePrice"]]])

In [1070]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,23.657512
1,c_LotArea,16.686685
5,c_GrLivArea,62.385682
8,c_GarageArea,10.107876
7,c_BedroomAbvGr,22.283992


In [1071]:
vif_feats=["c_GrLivArea"]

In [1072]:
X_train.drop(columns=vif_feats,inplace=True)
X_test.drop(columns=vif_feats,inplace=True)

In [1073]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,23.002075
1,c_LotArea,16.591958
7,c_GarageArea,9.249706
6,c_BedroomAbvGr,18.220271


In [1074]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [1075]:
vif_feats

['c_LotFrontage', 'c_LotArea', 'c_BedroomAbvGr']

In [1076]:
X_train.drop(columns=["c_LotFrontage"],inplace=True)
X_test.drop(columns=["c_LotFrontage"],inplace=True)

In [1077]:
X_train.columns

Index(['v_MSSubClass', 'v_MSZoning', 'c_LotArea', 'v_Street', 'v_LotShape',
       'v_LandContour', 'v_LotConfig', 'v_LandSlope', 'v_Condition1',
       'v_Condition2', 'v_BldgType', 'v_HouseStyle', 'v_OverallQual',
       'v_OverallCond', 'v_RoofStyle', 'v_RoofMatl', 'v_MasVnrType',
       'v_ExterQual', 'v_ExterCond', 'v_Foundation', 'v_BsmtQual',
       'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1', 'c_BsmtFinSF1',
       'v_BsmtFinType2', 'c_BsmtUnfSF', 'v_Heating', 'v_HeatingQC',
       'v_CentralAir', 'v_Electrical', 'c_2ndFlrSF', 'v_LowQualFinSF',
       'v_BsmtHalfBath', 'v_FullBath', 'c_HalfBath', 'c_BedroomAbvGr',
       'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional', 'v_Fireplaces',
       'v_GarageType', 'v_GarageFinish', 'c_GarageArea', 'v_GarageQual',
       'v_GarageCond', 'v_PavedDrive', 'c_WoodDeckSF', 'c_OpenPorchSF',
       'c_EnclosedPorch', 'c_MiscVal', 'tgt_SalePrice'],
      dtype='object')

In [1078]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif_feats=list(vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)["variables"][:10].values)

In [1079]:


vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotArea,10.268126
6,c_GarageArea,9.006685
5,c_BedroomAbvGr,15.667624


In [1080]:
X_train.drop(columns=["c_BedroomAbvGr"],inplace=True)
X_test.drop(columns=["c_BedroomAbvGr"],inplace=True)

In [1081]:
#BsmtCond: Estado general del sótano
#ExterCond: Estado actual del material en el exterior

In [1083]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])
vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)

,variables,VIF
5,c_GarageArea,8.816316
0,c_LotArea,7.625926


In [1084]:
vif

,variables,VIF
0,c_LotArea,7.625926
1,c_BsmtFinSF1,3.643785
2,c_BsmtUnfSF,4.941739
3,c_2ndFlrSF,3.083428
4,c_HalfBath,2.673205
5,c_GarageArea,8.816316
6,c_WoodDeckSF,1.635396
7,c_OpenPorchSF,1.818149
8,c_EnclosedPorch,1.213923
9,c_MiscVal,1.028192


#### MULTICOLINEALIDAD CATEGORICAS

In [1085]:
X_train

,v_MSSubClass,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice
0,90,rm,8472.0,grvl,otro,otro,corner,otro,otro,norm,otro,1story,5,5,gable,compshg,none,1,2,cblock,3.0,2.0,otro,otro,104.0,otro,0.0,gasa,2,n,sbrkr,0.0,0,0,1,0.0,1,2,typ,0,otro,unf,516.0,2.0,2.0,y,106.0,0.0,0.0,0.0,110000
1,50,rm,6240.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,6,6,gable,compshg,none,2,2,pconc,2.0,2.0,no,unf,0.0,unf,816.0,gasa,2,y,sbrkr,0.0,360,0,1,0.0,1,2,typ,1,detchd,unf,528.0,2.0,2.0,y,112.0,0.0,0.0,400.0,114500
2,160,otro,2544.0,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,7,5,gable,compshg,none,3,2,pconc,3.0,2.0,no,unf,0.0,unf,600.0,gasa,3,y,sbrkr,623.0,80,0,2,1.0,1,3,typ,0,detchd,rfn,480.0,2.0,2.0,y,0.0,166.0,0.0,0.0,147400
3,30,rl,11600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,5,gable,compshg,none,2,2,otro,1.0,2.0,no,unf,0.0,unf,700.0,gasa,3,y,sbrkr,0.0,0,0,1,0.0,1,1,typ,1,detchd,unf,252.0,2.0,1.0,y,0.0,0.0,67.0,0.0,137500
4,30,rm,9786.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,3,4,hip,compshg,none,2,2,otro,2.0,1.0,no,unf,0.0,unf,1007.0,gasa,1,n,sbrkr,0.0,0,0,1,0.0,1,2,typ,1,detchd,unf,210.0,2.0,1.0,p,0.0,100.0,48.0,0.0,91000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,70,rl,10592.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,7,hip,compshg,none,2,3,pconc,2.0,1.0,no,unf,0.0,unf,602.0,gasa,2,y,sbrkr,602.0,0,0,1,1.0,1,3,typ,2,detchd,unf,180.0,2.0,2.0,y,96.0,0.0,112.0,0.0,165000
760,60,rl,8750.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,5,gable,compshg,brkface,2,2,pconc,3.0,2.0,no,glq,505.0,unf,299.0,gasa,3,y,sbrkr,878.0,0,0,2,1.0,1,3,typ,0,attchd,rfn,523.0,2.0,2.0,y,0.0,77.0,0.0,0.0,200500
761,85,rl,10205.0,pave,otro,lvl,inside,gtl,norm,norm,1fam,otro,5,5,gable,compshg,none,2,2,cblock,2.0,2.0,otro,otro,784.0,unf,141.0,gasa,2,y,sbrkr,0.0,0,0,1,0.0,1,2,typ,0,attchd,unf,300.0,2.0,2.0,y,150.0,72.0,0.0,0.0,134500
762,60,rl,8935.0,pave,otro,lvl,inside,gtl,norm,norm,1fam,2story,7,5,gable,compshg,brkface,3,2,pconc,3.0,2.0,no,unf,0.0,unf,831.0,gasa,3,y,sbrkr,829.0,0,0,2,1.0,1,3,typ,0,attchd,rfn,493.0,2.0,2.0,y,144.0,68.0,0.0,0.0,195000


In [1086]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=False):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        #display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        #display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final


In [1087]:
v_feats=list(X_train.filter(like="v_").columns)

In [1088]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [1093]:
X_train["v_BldgType"].value_counts()

1fam    645
otro    119
Name: v_BldgType, dtype: int64

In [1096]:
X_train["v_OverallQual"].value_counts()

5     234
6     198
7     171
8      68
4      68
3      12
9      10
10      2
2       1
Name: v_OverallQual, dtype: int64

In [1097]:
X_train["v_OverallCond"].value_counts()

5    419
6    140
7    105
8     50
4     31
3      8
9      7
2      4
Name: v_OverallCond, dtype: int64

In [1095]:
X_train[["v_OverallQual","v_OverallCond"]].head(10)

,v_OverallQual,v_OverallCond
0,5,5
1,6,6
2,7,5
3,4,5
4,3,4
5,5,7
6,8,6
7,8,5
8,6,6
9,5,8


In [1089]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSSubClass,15.271982,1,3.907938,15.271982
v_MSZoning,2.261067,2,1.226248,1.503684
v_Street,1.322214,1,1.149876,1.322214
v_LotShape,1.364377,1,1.168065,1.364377
v_LandContour,1.526821,1,1.235646,1.526821
v_LotConfig,1.332973,2,1.074497,1.154544
v_LandSlope,1.558815,1,1.248525,1.558815
v_Condition1,1.280476,1,1.131581,1.280476
v_Condition2,1.519634,1,1.232734,1.519634


In [1098]:
X_train=X_train.drop(columns=["v_MSSubClass","v_OverallQual"])
X_test=X_test.drop(columns=["v_MSSubClass","v_OverallQual"])

In [1099]:
v_feats=list(X_train.filter(like="v_").columns)

In [1100]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [1101]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSZoning,2.190571,2,1.216576,1.480058
v_Street,1.271660,1,1.127679,1.271660
v_LotShape,1.351332,1,1.162468,1.351332
v_LandContour,1.463163,1,1.209613,1.463163
v_LotConfig,1.284080,2,1.064506,1.133172
v_LandSlope,1.525883,1,1.235266,1.525883
v_Condition1,1.251678,1,1.118784,1.251678
v_Condition2,1.471850,1,1.213198,1.471850
v_BldgType,1.673942,1,1.293809,1.673942


In [1102]:
X_train

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice
0,rm,8472.0,grvl,otro,otro,corner,otro,otro,norm,otro,1story,5,gable,compshg,none,1,2,cblock,3.0,2.0,otro,otro,104.0,otro,0.0,gasa,2,n,sbrkr,0.0,0,0,1,0.0,1,2,typ,0,otro,unf,516.0,2.0,2.0,y,106.0,0.0,0.0,0.0,110000
1,rm,6240.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,6,gable,compshg,none,2,2,pconc,2.0,2.0,no,unf,0.0,unf,816.0,gasa,2,y,sbrkr,0.0,360,0,1,0.0,1,2,typ,1,detchd,unf,528.0,2.0,2.0,y,112.0,0.0,0.0,400.0,114500
2,otro,2544.0,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,5,gable,compshg,none,3,2,pconc,3.0,2.0,no,unf,0.0,unf,600.0,gasa,3,y,sbrkr,623.0,80,0,2,1.0,1,3,typ,0,detchd,rfn,480.0,2.0,2.0,y,0.0,166.0,0.0,0.0,147400
3,rl,11600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,2,2,otro,1.0,2.0,no,unf,0.0,unf,700.0,gasa,3,y,sbrkr,0.0,0,0,1,0.0,1,1,typ,1,detchd,unf,252.0,2.0,1.0,y,0.0,0.0,67.0,0.0,137500
4,rm,9786.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,hip,compshg,none,2,2,otro,2.0,1.0,no,unf,0.0,unf,1007.0,gasa,1,n,sbrkr,0.0,0,0,1,0.0,1,2,typ,1,detchd,unf,210.0,2.0,1.0,p,0.0,100.0,48.0,0.0,91000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,rl,10592.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,7,hip,compshg,none,2,3,pconc,2.0,1.0,no,unf,0.0,unf,602.0,gasa,2,y,sbrkr,602.0,0,0,1,1.0,1,3,typ,2,detchd,unf,180.0,2.0,2.0,y,96.0,0.0,112.0,0.0,165000
760,rl,8750.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,5,gable,compshg,brkface,2,2,pconc,3.0,2.0,no,glq,505.0,unf,299.0,gasa,3,y,sbrkr,878.0,0,0,2,1.0,1,3,typ,0,attchd,rfn,523.0,2.0,2.0,y,0.0,77.0,0.0,0.0,200500
761,rl,10205.0,pave,otro,lvl,inside,gtl,norm,norm,1fam,otro,5,gable,compshg,none,2,2,cblock,2.0,2.0,otro,otro,784.0,unf,141.0,gasa,2,y,sbrkr,0.0,0,0,1,0.0,1,2,typ,0,attchd,unf,300.0,2.0,2.0,y,150.0,72.0,0.0,0.0,134500
762,rl,8935.0,pave,otro,lvl,inside,gtl,norm,norm,1fam,2story,5,gable,compshg,brkface,3,2,pconc,3.0,2.0,no,unf,0.0,unf,831.0,gasa,3,y,sbrkr,829.0,0,0,2,1.0,1,3,typ,0,attchd,rfn,493.0,2.0,2.0,y,144.0,68.0,0.0,0.0,195000


In [1103]:
X_train["v_LowQualFinSF"].value_counts()

0      754
80       2
360      2
234      1
420      1
384      1
390      1
513      1
144      1
Name: v_LowQualFinSF, dtype: int64

In [1104]:
##CREAR LAS VARIABLES DUMMY

In [890]:
from sklearn.preprocessing import OneHotEncoder

In [1105]:
dummy_vars=["v_MSZoning","v_Street","v_LotShape","v_LandContour","v_LotConfig","v_LandSlope","v_Condition1","v_Condition2","v_BldgType","v_HouseStyle",
"v_RoofStyle","v_MasVnrType","v_Foundation","v_BsmtExposure","v_BsmtFinType1","v_BsmtFinType2","v_Heating","v_CentralAir",
"v_Electrical","v_Functional","v_GarageType","v_GarageFinish","v_PavedDrive","v_RoofMatl","v_OverallCond"]

In [1106]:
X_train.shape

(764, 49)

In [1107]:
X_test.shape

(192, 49)

In [1108]:
X_test

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice
0,rl,13400,pave,reg,lvl,inside,gtl,norm,norm,1fam,otro,5,gable,compshg,brkface,2,2,cblock,2.0,2.0,otro,alq,516,otro,380,gasa,2,y,sbrkr,0,0,0,1,0,1,2,typ,1,attchd,rfn,484,2.0,2.0,y,0,0,0,0,159950
1,rl,34650,pave,reg,otro,inside,gtl,norm,norm,otro,1story,5,hip,compshg,none,2,2,cblock,2.0,2.0,otro,otro,1056,unf,0,gasa,2,n,sbrkr,0,0,0,1,0,1,2,typ,0,attchd,fin,572,2.0,2.0,y,264,0,0,0,145000
2,rl,7200,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,8,gable,compshg,none,2,2,cblock,2.0,2.0,no,alq,398,otro,317,gasa,3,y,sbrkr,0,0,0,1,0,1,3,typ,0,detchd,rfn,720,2.0,2.0,y,194,0,0,0,129000
3,rl,8123,pave,otro,lvl,inside,gtl,otro,norm,1fam,2story,5,gable,compshg,brkface,2,2,pconc,3.0,2.0,no,unf,0,unf,982,gasa,3,y,sbrkr,793,0,0,2,1,1,2,typ,1,attchd,fin,463,2.0,2.0,y,100,63,0,0,179000
4,rl,9858,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,6,gable,compshg,none,2,2,cblock,2.0,2.0,no,otro,510,unf,354,gasa,2,y,sbrkr,0,0,0,1,0,1,2,typ,0,attchd,rfn,288,2.0,2.0,y,33,0,0,600,130000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,rl,8250,pave,reg,lvl,inside,gtl,otro,norm,1fam,1story,5,hip,compshg,none,2,2,cblock,2.0,2.0,no,otro,250,otro,210,gasa,3,y,sbrkr,0,0,0,1,0,1,2,typ,1,attchd,unf,322,2.0,2.0,y,0,63,0,0,134000
188,rl,9600,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,hip,compshg,brkface,2,2,cblock,2.0,2.0,no,unf,0,unf,1164,gasa,2,y,sbrkr,0,0,0,1,1,1,2,typ,0,attchd,unf,528,2.0,2.0,y,0,0,0,0,140000
189,rl,14200,pave,reg,lvl,corner,gtl,norm,norm,1fam,2story,6,gable,compshg,brkface,2,2,cblock,2.0,2.0,no,otro,445,unf,479,gasa,3,y,sbrkr,941,0,0,2,1,1,3,typ,2,attchd,fin,487,2.0,2.0,y,105,66,0,0,226000
190,rm,3363,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,5,gable,compshg,otro,3,2,pconc,3.0,2.0,no,unf,0,unf,976,gasa,3,y,sbrkr,732,0,0,2,0,1,3,otro,0,detchd,unf,380,2.0,2.0,y,0,40,0,0,130000


In [1109]:
one=OneHotEncoder(drop="first",handle_unknown ="error")
#drop="first"

one.fit(X_train[dummy_vars])

replace_name=dict(zip(list(map(lambda x:f"x{str(x)}_",range(len(dummy_vars)))),dummy_vars))

features=one.get_feature_names()

features=list(map(lambda x:multiple_replace(x, replace_name),features))

X_train[features]=pd.DataFrame(one.transform(X_train[dummy_vars]).toarray(),columns=features)
X_test[features]=pd.DataFrame(one.transform(X_test[dummy_vars]).toarray(),columns=features)

X_train.drop(columns=dummy_vars,inplace=True)
X_test.drop(columns=dummy_vars,inplace=True)

In [1110]:
X_train

,c_LotArea,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,c_BsmtFinSF1,c_BsmtUnfSF,v_HeatingQC,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,c_GarageArea,v_GarageQual,v_GarageCond,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,8472.0,1,2,3.0,2.0,104.0,0.0,2,0.0,0,0,1,0.0,1,2,0,516.0,2.0,2.0,106.0,0.0,0.0,0.0,110000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,6240.0,2,2,2.0,2.0,0.0,816.0,2,0.0,360,0,1,0.0,1,2,1,528.0,2.0,2.0,112.0,0.0,0.0,400.0,114500,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2544.0,3,2,3.0,2.0,0.0,600.0,3,623.0,80,0,2,1.0,1,3,0,480.0,2.0,2.0,0.0,166.0,0.0,0.0,147400,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,11600.0,2,2,1.0,2.0,0.0,700.0,3,0.0,0,0,1,0.0,1,1,1,252.0,2.0,1.0,0.0,0.0,67.0,0.0,137500,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,9786.0,2,2,2.0,1.0,0.0,1007.0,1,0.0,0,0,1,0.0,1,2,1,210.0,2.0,1.0,0.0,100.0,48.0,0.0,91000,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,10592.0,2,3,2.0,1.0,0.0,602.0,2,602.0,0,0,1,1.0,1,3,2,180.0,2.0,2.0,96.0,0.0,112.0,0.0,165000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
760,8750.0,2,2,3.0,2.0,505.0,299.0,3,878.0,0,0,2,1.0,1,3,0,523.0,2.0,2.0,0.0,77.0,0.0,0.0,200500,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,10205.0,2,2,2.0,2.0,784.0,141.0,2,0.0,0,0,1,0.0,1,2,0,300.0,2.0,2.0,150.0,72.0,0.0,0.0,134500,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
762,8935.0,3,2,3.0,2.0,0.0,831.0,3,829.0,0,0,2,1.0,1,3,0,493.0,2.0,2.0,144.0,68.0,0.0,0.0,195000,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### PCA

In [1111]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

**Necesitamos normalizar los datos antes del análisis de componentes principales (PCA)**
- El PCA calcula una nueva proyección de su conjunto de datos. Y el nuevo eje se basa en la desviación estándar de sus variables. Por tanto, una variable con una desviación estándar alta tendrá un peso mayor para el cálculo del eje que una variable con una desviación estándar baja. Si normaliza sus datos, todas las variables tienen la misma desviación estándar, por lo que todas las variables tienen el mismo peso y su PCA calcula el eje relevante.

In [1112]:
X_train.filter(like="c_")

,c_LotArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_HalfBath,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal
0,8472.0,104.0,0.0,0.0,0.0,516.0,106.0,0.0,0.0,0.0
1,6240.0,0.0,816.0,0.0,0.0,528.0,112.0,0.0,0.0,400.0
2,2544.0,0.0,600.0,623.0,1.0,480.0,0.0,166.0,0.0,0.0
3,11600.0,0.0,700.0,0.0,0.0,252.0,0.0,0.0,67.0,0.0
4,9786.0,0.0,1007.0,0.0,0.0,210.0,0.0,100.0,48.0,0.0
...,...,...,...,...,...,...,...,...,...,...
759,10592.0,0.0,602.0,602.0,1.0,180.0,96.0,0.0,112.0,0.0
760,8750.0,505.0,299.0,878.0,1.0,523.0,0.0,77.0,0.0,0.0
761,10205.0,784.0,141.0,0.0,0.0,300.0,150.0,72.0,0.0,0.0
762,8935.0,0.0,831.0,829.0,1.0,493.0,144.0,68.0,0.0,0.0


In [1113]:
scaler = StandardScaler()
scaler.fit(X_train.filter(like="c_"))
df_train_sc=pd.DataFrame(scaler.transform(X_train.filter(like="c_")))
df_test_sc=pd.DataFrame(scaler.transform(X_test.filter(like="c_")))

In [1123]:
df_train_sc

,0,1,2,3,4,5,6,7,8,9
0,-0.128731,-0.719432,-1.470354,-0.801587,-0.720338,0.366998,0.315636,-0.740327,-0.394367,-0.112717
1,-0.826453,-0.991272,0.632087,-0.801587,-0.720338,0.432388,0.373908,-0.740327,-0.394367,4.050496
2,-1.981821,-0.991272,0.075559,0.820129,1.388237,0.170828,-0.713824,2.676635,-0.394367,-0.112717
3,0.849080,-0.991272,0.333211,-0.801587,-0.720338,-1.071580,-0.713824,-0.740327,0.719329,-0.112717
4,0.282024,-0.991272,1.124203,-0.801587,-0.720338,-1.300445,-0.713824,1.318083,0.403505,-0.112717
...,...,...,...,...,...,...,...,...,...,...
759,0.533980,-0.991272,0.080712,0.765465,1.388237,-1.463920,0.218518,-0.740327,1.467333,-0.112717
760,-0.041829,0.328723,-0.699974,1.483914,1.388237,0.405142,-0.713824,0.844649,-0.394367,-0.112717
761,0.413004,1.057987,-1.107064,-0.801587,-0.720338,-0.810021,0.742960,0.741728,-0.394367,-0.112717
762,0.016002,-0.991272,0.670735,1.356363,1.388237,0.241667,0.684688,0.659392,-0.394367,-0.112717


In [1114]:
df_train_sc.describe()

,0,1,2,3,4,5,6,7,8,9
count,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02
mean,1.627552e-16,-3.255104e-17,-1.627552e-17,4.650149e-18,-2.790089e-17,6.975223e-18,-3.720119e-17,5.115164e-17,2.325074e-18,3.487612e-17
std,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00
min,-2.370694e+00,-9.912724e-01,-1.470354e+00,-8.015873e-01,-7.203379e-01,-2.444769e+00,-7.138240e-01,-7.403274e-01,-3.943668e-01,-1.127174e-01
25%,-5.263577e-01,-9.912724e-01,-7.650312e-01,-8.015873e-01,-7.203379e-01,-7.664272e-01,-7.138240e-01,-7.403274e-01,-3.943668e-01,-1.127174e-01
50%,-2.909957e-03,-1.313153e-01,-1.073742e-01,-8.015873e-01,-7.203379e-01,6.184512e-02,-7.138240e-01,-3.698134e-01,-3.943668e-01,-1.127174e-01
75%,5.075649e-01,7.253746e-01,5.947279e-01,9.613460e-01,1.388237e+00,5.686170e-01,6.846883e-01,4.123827e-01,-3.943668e-01,-1.127174e-01
max,7.154210e+00,2.963485e+00,3.260139e+00,2.764627e+00,1.388237e+00,3.293197e+00,5.453227e+00,4.343947e+00,5.091000e+00,2.070335e+01


In [1115]:
for n_compo in range(1,11):
    print(f"N componentes : {n_compo}")
    pca = PCA(n_components=n_compo)
    pca.fit(df_train_sc)
    print(sum(pca.explained_variance_ratio_))
    print("\n")

N componentes : 1
0.19592803365844028


N componentes : 2
0.3731029282882202


N componentes : 3
0.508940235939656


N componentes : 4
0.61186623625348


N componentes : 5
0.709575253879116


N componentes : 6
0.7960753905672743


N componentes : 7
0.872895721468241


N componentes : 8
0.9395289572642904


N componentes : 9
0.9755411857468836


N componentes : 10
1.0000000000000002




In [1116]:
pca = PCA(n_components=7)
pca.fit(df_train_sc)

PCA(n_components=7)

In [1117]:
X_train_pca=pd.DataFrame(pca.transform(df_train_sc))
X_test_pca=pd.DataFrame(pca.transform(df_test_sc))

In [1118]:
#Porcentaje de varianza explicada por cada uno de los componentes seleccionados.
pca.explained_variance_ratio_

array([0.19592803, 0.17717489, 0.13583731, 0.102926  , 0.09770902,
       0.08650014, 0.07682033])

In [1119]:
sum(pca.explained_variance_ratio_)

0.8728957214682418

In [1122]:
#Para visualización creamos dos o tres componentes para tener una idea de como se ve nuestra información continua
#Tomando en cuenta la varianza explicada (algunas veces no es tan representativo)
pca = PCA(n_components=2)
dimention_2=pca.fit_transform(df_train_sc)
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter(dimention_2, x=0, y=1,title=f'Total de Varianza Explicada: {total_var:.2f}%',
                 labels={'0': 'Componente Principal 1', '1': 'Componente Principal 2'})
fig.show()

In [1124]:
pca = PCA(n_components=3)
dimention_3=pca.fit_transform(df_train_sc)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    dimention_3, x=0, y=1, z=2, 
    title=f'Total de Varianza Explicada: {total_var:.2f}%',
    labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'}
)
fig.show()

### K- Best

Score function:
- For regression: f_regression, mutual_info_regression
- For classification: chi2, f_classif, mutual_info_classif

**f_regression**
- Pruebas de regresión lineal univariante.

- Modelo lineal para probar el efecto individual de cada uno de los muchos regresores. Esta es una función de puntuación que se utilizará en un procedimiento de selección de características

- Esto se hace en 2 pasos:

- Se calcula la correlación entre cada regresor y el objetivo, es decir, ((X [:, i] - mean (X [:, i])) * (y - mean_y)) / (std (X [:, i] ) * estándar (y)).

- Se convierte en una puntuación F y luego en un valor p.

- mutual_info_classif : para discretas y continuas
- f_regressión : solo continuas

https://towardsdatascience.com/fisher-test-for-regression-analysis-1e1687867259

** Estime la información mutua para una variable objetivo discreta.

** La información mutua (MI) [1] entre dos variables aleatorias es un valor no negativo, que mide la dependencia entre las variables. Es igual a cero si y solo si dos variables aleatorias son independientes, y los valores más altos significan una mayor dependencia.

** La función se basa en métodos no paramétricos basados ​​en la estimación de la entropía a partir de distancias de los vecinos más cercanos k.

In [914]:
X=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_")
y=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

In [915]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif 

In [920]:
kb = SelectKBest(mutual_info_classif , k=5)

In [921]:
X

,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_HalfBath,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal
0,6240.0,0.0,0.0,816.0,0.0,0.0,528.0,112.0,0.0,0.0,400.0
1,7742.0,0.0,763.0,192.0,0.0,0.0,386.0,0.0,0.0,0.0,0.0
2,9600.0,0.0,442.0,625.0,0.0,0.0,436.0,290.0,0.0,0.0,0.0
3,11600.0,0.0,0.0,700.0,0.0,0.0,252.0,0.0,0.0,67.0,0.0
4,10206.0,0.0,0.0,0.0,0.0,0.0,528.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
759,21286.0,0.0,0.0,720.0,551.0,0.0,312.0,0.0,0.0,108.0,0.0
760,8834.0,216.0,1170.0,292.0,762.0,1.0,738.0,184.0,0.0,0.0,0.0
761,7094.0,0.0,180.0,340.0,0.0,0.0,384.0,0.0,0.0,0.0,0.0
762,2645.0,456.0,0.0,776.0,677.0,1.0,492.0,206.0,0.0,0.0,0.0


In [922]:
X_new=kb.fit_transform(X, y)

In [923]:
cols=X.columns[kb.get_support()]

In [924]:
pd.DataFrame(X_new,columns=cols)

,c_LotArea,c_MasVnrArea,c_HalfBath,c_GarageArea,c_OpenPorchSF
0,6240.0,0.0,0.0,528.0,0.0
1,7742.0,0.0,0.0,386.0,0.0
2,9600.0,0.0,0.0,436.0,0.0
3,11600.0,0.0,0.0,252.0,0.0
4,10206.0,0.0,0.0,528.0,0.0
...,...,...,...,...,...
759,21286.0,0.0,0.0,312.0,0.0
760,8834.0,216.0,1.0,738.0,0.0
761,7094.0,0.0,0.0,384.0,0.0
762,2645.0,456.0,1.0,492.0,0.0


In [925]:
scores=pd.DataFrame(kb.scores_)
scores["Variable"]=X.columns
scores.rename(columns={0:"Score"},inplace=True)

In [926]:
scores=scores.sort_values(by="Score",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [927]:
scores

,Score,Variable
0,0.335697,c_HalfBath
1,0.249196,c_GarageArea
2,0.195370,c_OpenPorchSF
3,0.164919,c_LotArea
4,0.117103,c_MasVnrArea
5,0.074608,c_WoodDeckSF
6,0.065104,c_MiscVal
7,0.055629,c_2ndFlrSF
8,0.044195,c_BsmtUnfSF
9,0.030172,c_BsmtFinSF1


In [928]:
X[list(scores[:20]["Variable"].values)][cols]

,c_LotArea,c_MasVnrArea,c_HalfBath,c_GarageArea,c_OpenPorchSF
0,6240.0,0.0,0.0,528.0,0.0
1,7742.0,0.0,0.0,386.0,0.0
2,9600.0,0.0,0.0,436.0,0.0
3,11600.0,0.0,0.0,252.0,0.0
4,10206.0,0.0,0.0,528.0,0.0
...,...,...,...,...,...
759,21286.0,0.0,0.0,312.0,0.0
760,8834.0,216.0,1.0,738.0,0.0
761,7094.0,0.0,0.0,384.0,0.0
762,2645.0,456.0,1.0,492.0,0.0


## VarClus

- Varclus es un buen algoritmo de reducción de dimensiones. Aquí hay una breve descripción:
- Se elige un grupo para dividir.
- El grupo elegido se divide en dos grupos al encontrar los dos primeros componentes principales, realizar una rotación ortoblicua y asignar cada variable al componente girado con el que tiene la correlación al cuadrado más alto.
- Las variables se reasignan iterativamente a los conglomerados para maximizar la varianza contabilizada por los componentes del conglomerado

In [1126]:
from varclushi import VarClusHi

In [1133]:
vc = VarClusHi(X,maxeigval2=.7)

In [1134]:
vc.varclus()

In [1135]:
vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_EnclosedPorch,1.000000,0.033443,0.000000
1,1,c_BsmtFinSF1,0.786901,0.039508,0.221865
2,1,c_BsmtUnfSF,0.786901,0.037686,0.221445
3,2,c_2ndFlrSF,0.821109,0.031683,0.184744
4,2,c_HalfBath,0.821109,0.037034,0.185771
5,3,c_WoodDeckSF,1.000000,0.043775,0.000000
6,4,c_MiscVal,1.000000,0.004830,0.000000
7,5,c_GarageArea,1.000000,0.064245,0.000000
8,6,c_LotArea,1.000000,0.049686,0.000000
9,7,c_OpenPorchSF,1.000000,0.062331,0.000000


In [1137]:
vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_EnclosedPorch,1.000000,0.033443,0.000000
2,1,c_BsmtUnfSF,0.786901,0.037686,0.221445
3,2,c_2ndFlrSF,0.821109,0.031683,0.184744
5,3,c_WoodDeckSF,1.000000,0.043775,0.000000
6,4,c_MiscVal,1.000000,0.004830,0.000000
7,5,c_GarageArea,1.000000,0.064245,0.000000
8,6,c_LotArea,1.000000,0.049686,0.000000
9,7,c_OpenPorchSF,1.000000,0.062331,0.000000
10,8,c_MasVnrArea,1.000000,0.064245,0.000000


In [1138]:
columns=list(vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")["Variable"].values)

In [1139]:
columns

['c_EnclosedPorch',
 'c_BsmtUnfSF',
 'c_2ndFlrSF',
 'c_WoodDeckSF',
 'c_MiscVal',
 'c_GarageArea',
 'c_LotArea',
 'c_OpenPorchSF',
 'c_MasVnrArea']

## Arboles de decision

In [1140]:
x_train=X_train[[x for x in X_train if x!="tgt_SalePrice"]]
y_train=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

x_test=X_test[[x for x in X_test if x!="tgt_SalePrice"]]
y_test=X_test[[x for x in X_test if x=="tgt_SalePrice"]]

In [1141]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [1142]:

model = DecisionTreeRegressor()

model.fit(x_train, y_train)

importance = model.feature_importances_

In [1144]:
pred_test=model.predict(x_test)
pred_train=model.predict(x_train)

In [1145]:
r2_score(y_test,pred_test)

0.6989273028429442

In [1146]:
r2_score(y_train,pred_train)

0.9999852187046931

In [1147]:
importance

array([4.18103789e-02, 4.36254336e-01, 3.15006835e-03, 8.80264421e-03,
       6.53011674e-04, 1.29870031e-01, 1.14560603e-02, 3.54091831e-03,
       3.42749154e-02, 3.26916675e-04, 3.14964113e-04, 7.05767053e-02,
       1.37512445e-02, 1.51362701e-06, 8.36091745e-04, 2.62773738e-03,
       8.58508651e-02, 1.94896863e-03, 3.31009712e-04, 1.10538852e-02,
       2.49457530e-02, 5.42280353e-03, 2.23829565e-05, 1.22720956e-03,
       2.01817710e-03, 0.00000000e+00, 1.11280634e-03, 1.90569020e-04,
       1.27640821e-03, 6.88829755e-04, 4.03771216e-04, 1.85608944e-03,
       0.00000000e+00, 2.21599040e-02, 3.01903590e-03, 9.28453355e-04,
       1.52346138e-03, 2.97465201e-03, 4.90687953e-05, 2.61027638e-03,
       5.19249246e-03, 8.56935570e-04, 2.14210405e-03, 1.64135924e-04,
       1.19267648e-03, 4.96978287e-03, 3.98911184e-03, 7.31931164e-04,
       0.00000000e+00, 2.30210495e-02, 0.00000000e+00, 6.76496613e-04,
       5.34938878e-04, 4.02385653e-05, 2.67059568e-03, 2.70406655e-04,
      

In [1151]:
scores=pd.DataFrame()
scores["gini_index"]=importance
scores["feature"]=x_train.columns

In [1152]:
scores=scores.sort_values(by="gini_index",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [1153]:
feats=list(scores[:10]["feature"].values)

In [1154]:
scores

,gini_index,feature
0,0.436254,v_ExterQual
1,0.129870,c_BsmtFinSF1
2,0.085851,c_GarageArea
3,0.070577,v_FullBath
4,0.041810,c_LotArea
...,...,...
63,0.000000,v_Heating_otro
64,0.000000,v_RoofMatl_otro
65,0.000000,v_Condition2_otro
66,0.000000,v_Electrical_otro
